In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pprint
import sys
import astropy.io.fits as pyfits
import MockImageFunctions
from matplotlib.colors import LogNorm
import subprocess
import os 
import shutil 
from scipy import ndimage
import traceback

original skirt (200, 500, 500)
1.65308596 arcsec


MUST change redshift in all paths or you end up saving the same galaxy under multiple names


In [2]:
# %matplotlib inline
# testim = np.load('/n/holystore01/LABS/hernquist_lab/Users/aschechter/fullmocks/z1/allfilters478489_1.npy')
# testimsum = np.sum(testim, axis = 0)
# # print(np.shape(np.sum(testim, axis = 0)))
# # print(np.min(testim))
# # print(np.min(np.sum(testim, axis = 0)))
# # print(np.log10(np.min(np.sum(testim, axis = 0))))
# # print(np.max(testim))
# # print(np.max(np.sum(testim, axis = 0)))
# # print(np.log10(np.max(np.sum(testim, axis = 0))))

# print(np.nanmin(np.log10(testimsum)))
# print(np.nanmin(testimsum))

# plt.figure()
# plt.imshow(testim[3], norm = LogNorm())
# plt.colorbar()
# plt.show()

# plt.figure()
# plt.imshow(testimsum, norm = LogNorm())
# plt.colorbar()
# plt.show()


#print(np.min(testim - np.min(testim)))
# print(np.shape(testim[0]))
# plt.figure()
# plt.imshow(abs(np.sum(testim - np.min(testim), axis = 2))/np.max(testim - np.min(testim)), norm = LogNorm())
# plt.colorbar()

# testim = testim - np.min(testim)
# testimnorm = testim/np.max(testim)
# print(np.min(testimnorm), np.max(testimnorm))
# print(np.shape(testimnorm))

# plt.figure()
# plt.imshow(abs(np.sum(testimnorm, axis=2)))
# plt.colorbar()

In [2]:
#read in subhalos, viewpoints, RA and Dec of backgrounds, and mock images
SandV40 = pd.read_csv('SubhaloList_WithBackgroundCoords_ForMakeMocks40.csv')
SandV50 = pd.read_csv('SubhaloListForMakeMocks50.csv')
b = np.loadtxt('BackgroundsUsed.txt', dtype = str)
print(len(b))
b_nofits = []
for bb in range(len(b)):
    #print(bb)
    b_nofits.append(b[bb][:-5])
    #SandV50
#print(len(b_nofits))
print(b_nofits)
SandV50['RA Dec'] = b_nofits

3120
['150.1283899966_2.3811474863', '150.2021745266_2.4605269478', '150.1759498292_2.4398787738', '150.1366122455_2.2725145984', '150.187536607_2.2943629292', '150.1785604939_2.2398801119', '150.1948949578_2.3413123119', '150.1738405649_2.2486636302', '150.1544979211_2.4662457077', '150.2028880262_2.3947613333', '150.1555987417_2.4591458242', '150.1397482018_2.2826312007', '150.1319762019_2.3613976218', '150.1842252231_2.4685611401', '150.190428793_2.4260616237', '150.1612535096_2.4465125582', '150.1456378626_2.3671140217', '150.2011798158_2.276412241', '150.1827394419_2.4441450453', '150.1933969371_2.4036951192', '150.1410322546_2.2646312064', '150.1806143252_2.2924299292', '150.1221515947_2.4549131389', '150.1977938708_2.277345767', '150.1854362622_2.3231462478', '150.1243865706_2.3833141609', '150.1229182699_2.299381338', '150.1979375948_2.465210445', '150.1946987052_2.4157448764', '150.1539442734_2.3416973378', '150.1879424216_2.4087286816', '150.1672020697_2.2469972112', '150.190

In [3]:
SandV = pd.concat([SandV40, SandV50], ignore_index = True)
SandV

,Unnamed: 0.1,Unnamed: 0,Subfind_ID,Viewpoint,Type,RA Dec
0,0.0,0,289514,1,Descendant,_150.1608837615_2.3545137154
1,1.0,1,289514,2,Descendant,_150.120166145_2.3333478826
2,2.0,2,289514,3,Descendant,_150.1300249081_2.4001305995
3,3.0,3,289514,4,Descendant,_150.1754419857_2.2534468944
4,4.0,4,289514,5,Descendant,_150.1745470762_2.401579412
...,...,...,...,...,...,...
8467,NaN,3115,118843,2,0.0,150.1823311413_2.2648465818
8468,NaN,3116,118843,3,0.0,150.1554634443_2.3889802363
8469,NaN,3117,118843,4,0.0,150.1273386521_2.3213312345
8470,NaN,3118,118843,5,0.0,150.1541771043_2.314264134


In [5]:

bpath ='/n/holystore01/LABS/hernquist_lab/Users/aschechter/background_cutouts/z1/'
mockspath = '/n/holystore01/LABS/hernquist_lab/Users/aschechter/mocks_nobackground_z1/'

SandV

#type = 0.0 is a non merger

,Unnamed: 0.1,Unnamed: 0,Subfind_ID,Viewpoint,Type,RA Dec
0,0.0,0,289514,1,Descendant,_150.1608837615_2.3545137154
1,1.0,1,289514,2,Descendant,_150.120166145_2.3333478826
2,2.0,2,289514,3,Descendant,_150.1300249081_2.4001305995
3,3.0,3,289514,4,Descendant,_150.1754419857_2.2534468944
4,4.0,4,289514,5,Descendant,_150.1745470762_2.401579412
...,...,...,...,...,...,...
8467,NaN,3115,118843,2,0.0,150.1823311413_2.2648465818
8468,NaN,3116,118843,3,0.0,150.1554634443_2.3889802363
8469,NaN,3117,118843,4,0.0,150.1273386521_2.3213312345
8470,NaN,3118,118843,5,0.0,150.1541771043_2.314264134


In [12]:
### trying to make sure sets are super even between mergers and nonmergers
### need to make split in each class
#SubIDs_norepeats = [*set(SandV['Subfind_ID'])]
SandV_mergers = SandV[SandV['Type'] != '0.0']
print(len(SandV_mergers))

SandV_nonmergers = SandV[SandV['Type'] == '0.0']
print(len(SandV_nonmergers))

4236
4236


In [14]:
SubIDs_norepeats_mergers = [*set(SandV_mergers['Subfind_ID'])]
print('len SubIDs ', len(SubIDs_norepeats_mergers))
SubIDs_norepeats_nonmergers = [*set(SandV_nonmergers['Subfind_ID'])]
print('len SubIDs ', len(SubIDs_norepeats_nonmergers))

test_set_IDs_mergers = np.random.choice(SubIDs_norepeats_mergers, size = int(len(SubIDs_norepeats_mergers)*0.15), replace = False)
test_set_IDs_nonmergers = np.random.choice(SubIDs_norepeats_nonmergers, size = int(len(SubIDs_norepeats_nonmergers)*0.15), replace = False)
print('test ', len(test_set_IDs_mergers), len(test_set_IDs_nonmergers))
trainval_mergers = []
trainval_nonmergers = []
why = 0
for t in SubIDs_norepeats_mergers:
    #print('ID = ', t)
    if t in test_set_IDs_mergers:
        why +=1
    else: 
        trainval_mergers.append(t)
        
for t in SubIDs_norepeats_nonmergers:
    #print('ID = ', t)
    if t in test_set_IDs_nonmergers:
        why +=1
    else: 
        trainval_nonmergers.append(t)
trainval_mergers = np.array(trainval_mergers)
trainval_nonmergers = np.array(trainval_nonmergers)
print('trainval ', len(trainval_mergers), len(trainval_nonmergers))
print(len(SubIDs_norepeats_mergers) - len(test_set_IDs_mergers))
print(why)

len SubIDs  705
len SubIDs  706
test  105 105
trainval  600 601
600
210


In [16]:
val_set_IDs_mergers = np.random.choice(trainval_mergers, size = int(len(test_set_IDs_mergers)), replace = False)
val_set_IDs_nonmergers = np.random.choice(trainval_nonmergers, size = int(len(test_set_IDs_nonmergers)), replace = False)

train_set_IDs_mergers = []
train_set_IDs_nonmergers = []
why2 = 0 
for v in trainval_mergers:
    if v in val_set_IDs_mergers:
        why2 +=1
    else:
        train_set_IDs_mergers.append(v)
train_set_IDs_mergers = np.array(train_set_IDs_mergers)

why3 = 0 
for v in trainval_nonmergers:
    if v in val_set_IDs_nonmergers:
        why3 +=1
    else:
        train_set_IDs_nonmergers.append(v)
train_set_IDs_nonmergers = np.array(train_set_IDs_nonmergers)

print(len(test_set_IDs_mergers), len(val_set_IDs_mergers), len(train_set_IDs_mergers))
print(len(test_set_IDs_nonmergers), len(val_set_IDs_nonmergers), len(train_set_IDs_nonmergers))

105 105 495
105 105 496


In [17]:
SetAssignment = np.zeros(len(SandV['Subfind_ID']), dtype=object)
for i in range(len(SandV['Subfind_ID'])):
    if SandV['Subfind_ID'][i] in train_set_IDs_mergers:
        SetAssignment[i] = 'Training'
    elif SandV['Subfind_ID'][i] in val_set_IDs_mergers:
        SetAssignment[i] = 'Validation'
    elif SandV['Subfind_ID'][i] in test_set_IDs_mergers:
        SetAssignment[i] = 'Test'
    elif SandV['Subfind_ID'][i] in train_set_IDs_nonmergers:
        SetAssignment[i] = 'Training'
    elif SandV['Subfind_ID'][i] in val_set_IDs_nonmergers:
        SetAssignment[i] = 'Validation'
    elif SandV['Subfind_ID'][i] in test_set_IDs_nonmergers:
        SetAssignment[i] = 'Test'
np.set_printoptions(threshold=sys.maxsize)
print(len(SetAssignment))
SandV['Set Assignment'] = SetAssignment

8472


In [18]:
SandV
CNNMocksInfo = SandV.copy()
CNNMocksInfo.to_csv('CNNMocksInfo_z1.csv', index = False)
print(len(CNNMocksInfo))

8472


In [19]:
CNNMocksInfo

,Unnamed: 0.1,Unnamed: 0,Subfind_ID,Viewpoint,Type,RA Dec,Set Assignment
0,0.0,0,289514,1,Descendant,_150.1608837615_2.3545137154,Validation
1,1.0,1,289514,2,Descendant,_150.120166145_2.3333478826,Validation
2,2.0,2,289514,3,Descendant,_150.1300249081_2.4001305995,Validation
3,3.0,3,289514,4,Descendant,_150.1754419857_2.2534468944,Validation
4,4.0,4,289514,5,Descendant,_150.1745470762_2.401579412,Validation
...,...,...,...,...,...,...,...
8467,NaN,3115,118843,2,0.0,150.1823311413_2.2648465818,Training
8468,NaN,3116,118843,3,0.0,150.1554634443_2.3889802363,Training
8469,NaN,3117,118843,4,0.0,150.1273386521_2.3213312345,Training
8470,NaN,3118,118843,5,0.0,150.1541771043_2.314264134,Training


In [11]:
# tstack = np.load(fullmockspath + 'full_577921_5allfilters.npy')
# np.shape(tstack)
# #print(tstack)

In [20]:
fullmockspath1 =  '/n/holystore01/LABS/hernquist_lab/Users/aschechter/fullmocks/z1/'
fullmockspath15 =  '/n/holystore01/LABS/hernquist_lab/Users/aschechter/fullmocks/z15/'
#fullmockslist = os.listdir(fullmockspath)
#print(fullmockslist[0])

In [21]:
for index, row in CNNMocksInfo.iterrows():
    subid = row['Subfind_ID']; viewpoint = row['Viewpoint']; classification = row['Type']; setassignment = row['Set Assignment']
    if os.path.exists(fullmockspath1 + 'allfilters' + str(subid) + '_' + str(viewpoint) + '.npy'):
        fullmockspath = fullmockspath1
    elif os.path.exists(fullmockspath15 + 'allfilters' + str(subid) + '_' + str(viewpoint) + '.npy'):
        fullmockspath = fullmockspath15
    else:
        print(str(subid) + 'does not work')
    
    try:
        source = fullmockspath + 'allfilters' + str(subid) + '_' + str(viewpoint) + '.npy'
        if setassignment == 'Training': 
            destination = "/n/holystore01/LABS/hernquist_lab/Users/aschechter/z1widebinmocks/training/"
            if classification == '0.0':
                shutil.copy(source, destination + 'nonmergers/')
            elif classification == 'Merger':
                shutil.copy(source, destination + 'mergers/')
                shutil.copy(source, destination + 'anymergers/')
            elif classification == 'Descendant':
                shutil.copy(source, destination + 'latemergers/')
                shutil.copy(source, destination + 'anymergers/')
            elif classification == 'first_progenitor' or classification == 'next_progenitor':
                shutil.copy(source, destination + 'earlymergers/')
                shutil.copy(source, destination + 'anymergers/')

        elif setassignment == 'Validation': 
            destination = "/n/holystore01/LABS/hernquist_lab/Users/aschechter/z1widebinmocks/validation/"
            if classification == '0.0':
                shutil.copy(source, destination + 'nonmergers/')
            elif classification == 'Merger':
                shutil.copy(source, destination + 'mergers/')
                shutil.copy(source, destination + 'anymergers/')
            elif classification == 'Descendant':
                shutil.copy(source, destination + 'latemergers/')
                shutil.copy(source, destination + 'anymergers/')
            elif classification == 'first_progenitor' or classification == 'next_progenitor':
                shutil.copy(source, destination + 'earlymergers/')
                shutil.copy(source, destination + 'anymergers/')
        elif setassignment == 'Test':
            destination = "/n/holystore01/LABS/hernquist_lab/Users/aschechter/z1widebinmocks/test/"
            if classification == '0.0':
                shutil.copy(source, destination + 'nonmergers/')
            elif classification == 'Merger':
                shutil.copy(source, destination + 'mergers/')
                shutil.copy(source, destination + 'anymergers/')
            elif classification == 'Descendant':
                shutil.copy(source, destination + 'latemergers/')
                shutil.copy(source, destination + 'anymergers/')
            elif classification == 'first_progenitor' or classification == 'next_progenitor':
                shutil.copy(source, destination + 'earlymergers/')
                shutil.copy(source, destination + 'anymergers/')
        print('classified successfully')
    except Exception as e:
            # Print error and traceback
            print(f"Error processing Subfind_ID {subid} and Viewpoint {viewpoint}:")
            print(f"Exception: {e}")
            traceback.print_exc()  # This will give you a detailed stack trace

classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified succe

Traceback (most recent call last):
  File "/tmp/ipykernel_1004855/3319999823.py", line 15, in <cell line: 1>
    shutil.copy(source, destination + 'nonmergers/')
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 418, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 264, in copyfile
    with open(src, 'rb') as fsrc, open(dst, 'wb') as fdst:
FileNotFoundError: [Errno 2] No such file or directory: '/n/holystore01/LABS/hernquist_lab/Users/aschechter/fullmocks/z15/allfilters15216_1.npy'
Traceback (most recent call last):
  File "/tmp/ipykernel_1004855/3319999823.py", line 15, in <cell line: 1>
    shutil.copy(source, destination + 'nonmergers/')
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 418, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 264, in c

classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified succe

Traceback (most recent call last):
  File "/tmp/ipykernel_1004855/3319999823.py", line 15, in <cell line: 1>
    shutil.copy(source, destination + 'nonmergers/')
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 418, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 264, in copyfile
    with open(src, 'rb') as fsrc, open(dst, 'wb') as fdst:
FileNotFoundError: [Errno 2] No such file or directory: '/n/holystore01/LABS/hernquist_lab/Users/aschechter/fullmocks/z15/allfilters136_1.npy'
Traceback (most recent call last):
  File "/tmp/ipykernel_1004855/3319999823.py", line 15, in <cell line: 1>
    shutil.copy(source, destination + 'nonmergers/')
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 418, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 264, in cop

classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified succe

Traceback (most recent call last):
  File "/tmp/ipykernel_1004855/3319999823.py", line 15, in <cell line: 1>
    shutil.copy(source, destination + 'nonmergers/')
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 418, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 264, in copyfile
    with open(src, 'rb') as fsrc, open(dst, 'wb') as fdst:
FileNotFoundError: [Errno 2] No such file or directory: '/n/holystore01/LABS/hernquist_lab/Users/aschechter/fullmocks/z15/allfilters25874_1.npy'
Traceback (most recent call last):
  File "/tmp/ipykernel_1004855/3319999823.py", line 15, in <cell line: 1>
    shutil.copy(source, destination + 'nonmergers/')
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 418, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 264, in c

classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified succe

Traceback (most recent call last):
  File "/tmp/ipykernel_1004855/3319999823.py", line 15, in <cell line: 1>
    shutil.copy(source, destination + 'nonmergers/')
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 418, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 264, in copyfile
    with open(src, 'rb') as fsrc, open(dst, 'wb') as fdst:
FileNotFoundError: [Errno 2] No such file or directory: '/n/holystore01/LABS/hernquist_lab/Users/aschechter/fullmocks/z15/allfilters119_1.npy'
Traceback (most recent call last):
  File "/tmp/ipykernel_1004855/3319999823.py", line 15, in <cell line: 1>
    shutil.copy(source, destination + 'nonmergers/')
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 418, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/n/home09/aschechter/.conda/envs/py38/lib/python3.8/shutil.py", line 264, in cop

classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified successfully
classified succe

In [22]:
lab = np.load('/n/holystore01/LABS/hernquist_lab/Users/aschechter/z15mocks/training/anymergers/mergerlabel.npy')
print(lab)

FileNotFoundError: [Errno 2] No such file or directory: '/n/holystore01/LABS/hernquist_lab/Users/aschechter/z15mocks/training/anymergers/mergerlabel.npy'

In [23]:
for assignedset in ['training', 'test', 'validation']:
        for t in ['mergers', 'anymergers', 'earlymergers', 'latemergers']:
                length = len(glob.glob('/n/holystore01/LABS/hernquist_lab/Users/aschechter/z1widebinmocks/' + assignedset + '/' + t + '/allfilters*.npy'))
                print(length)
                mergerlabels = np.repeat(0, length)
                #print(mergerlabels)
                np.save('/n/holystore01/LABS/hernquist_lab/Users/aschechter/z1widebinmocks/' + assignedset + '/' + t + '/mergerlabel.npy', mergerlabels)

654
2970
1374
942
120
630
270
240
138
630
246
246


In [24]:
for assignedset in ['training', 'test', 'validation']:
    length = len(glob.glob('/n/holystore01/LABS/hernquist_lab/Users/aschechter/z1widebinmocks/' + assignedset +'/nonmergers/allfilters*.npy'))
    print(length)
    mergerlabels = np.repeat(1, length)
    #print(mergerlabels)
    np.save('/n/holystore01/LABS/hernquist_lab/Users/aschechter/z1widebinmocks/' + assignedset +'/nonmergers/mergerlabel.npy', mergerlabels)

2958
630
624


In [17]:
z1w = np.load('/n/holystore01/LABS/hernquist_lab/Users/aschechter/z1widebinmocks/training/anymergers/allfilters451708_2.npy')
print(np.shape(z1w))
z1 = np.load('/n/holystore01/LABS/hernquist_lab/Users/aschechter/z15mocks/training/anymergers/allfilters289098_2.npy')

z1w = np.transpose(z1w, (2,1,0))
print(np.shape(z1w))
print(np.shape(z1))

(202, 202, 4)


FileNotFoundError: [Errno 2] No such file or directory: '/n/holystore01/LABS/hernquist_lab/Users/aschechter/z15mocks/training/anymergers/allfilters289098_2.npy'

In [ ]:
a=[1,2]
b=[1,2]
c=[1,2]
d=[1,2]

e = np.array([a,b, c])
print(np.shape(e.T))
e2 = np.array([e, d])
print(e2)

In [ ]:
testmergers = CNNMocksInfo.loc[(CNNMocksInfo['Set Assignment'] == 'Test') & (CNNMocksInfo['Type'] == 'Merger')]


In [ ]:
for index, row in testmergers.iterrows():
    subid = row['Subfind_ID']; viewpoint = row['Viewpoint']; classification = row['Type']; setassignment = row['Set Assignment']
    rfilter =  '/n/holystore01/LABS/hernquist_lab/Users/aschechter/z15mocks/test/mergers' + 'full_' + str(subid) + '_' + str(viewpoint)  + '*.npy'
    sourcelist = glob.glob(source)
    print(len(sourcelist))
    
    